
# Django Forms
[see](https://docs.djangoproject.com/en/2.1/topics/forms/)

## Content
 - [The Basics](#The-Basic)
 - [More about Django Forms](#More-about-Django-Form-Class)
 - [Working with Form Template](#Working-with-form-templates)
     - [Form rendering options](#Form-rendering-option)
     - [Rendering Fields Manually](#Rendering-Fields-Manually:)
     - [Rendering Form's error message](#Rendering-form's-error-message)
     - [Looping over Form's Field](#Looping-over-form's-fields)
 - [Core Field Arguments](#Core-Field-Arguments)
     - [required](#required)
     - [label](#label)
     - [initial](#initial)
     - [help_text](#help_text)
     - [error_messages](#error_messages)
     - [validators](#validators)
     - [localize](#localize)
     - [disabled](#disabled)
 - [Widgets](#Widgets)
     - [Widgets inheriting from the select widget](#Widgets-inheriting-from-the-Select-widget)
     - [Styling widget instances](#Styling-widget-instances)
     - [Multi Widget](#Multi-Widget)
 - [Creating Forms From Models](#Creating-Forms-from-Model)
     - [A full example](#A-full-example:)
     - [The save() method](#The-save()-method)
     - [Selecting Fields to Use](#Selecting-Fields-to-Use)
     - [Overriding the default fields](#Overriding-the-default-fields)
     - [Providing initial values](#Providing-initial-values)
     - [ModelForm factory function](#ModelForm-factory-function)
 - [Formsets](#Formsets)
     - [Using initial data with formsets](#Using-initial-data-with-formsets)
     - [Custom Formset Validation](#Custom-Formset-Validation)
     - [can_delete](#can_delete)
     - [Adding additional fields to a formset](#Adding-additional-fields-to-a-formset)
     - [Using a formset in views and templates](#Using-a-formset-in-views-and-templates)
     - [Manually rendered can_delete and can_order](#Manually-rendered-can_delete-and-can_order)
     - [Using more than one formset in a view](#Using-more-than-one-formset-in-a-view)
 - [Model Formsets](#Model-Formsets)
     - [Changing the queryset](#Changing-the-queryset)
     - [Changing the form](#Changing-the-form)
     - [Specifying widgets to use in the form with widgets](#Specifying-widgets-to-use-in-the-form-with-widgets)
     - [Saving objects in the formset](#Saving-objects-in-the-formset)
     - [Using a model formset in a view](#Using-a-model-formset-in-a-view)
 
## The Basic

**forms.py**
``` Python
from django import forms

Class NameForm(forms.Form):
    name = forms.CharField(label='Your Name', max_lenght=100)
    
```

The whole form when rendered will look like:

``` html
<label for='your_name'>Your Name: </label>
<input id='your_name' name='your_name' type='text' maxlength=100 required>
```

Note that it doesn't include form tag or submit button, we have to provide that by ourselves.

**views.py**
``` Python
from django.http import HttpResponseRedirect
from django.shortcuts import render

from .forms import NameForm

def get_name(request):
    # if this is a post data we need to process that
    if request.method=='POST':
        # create a form instance and populate with form data
        form = NameForm(request.POST)
        # check whether form is valid or not
        if form.is_valid():
            # process the data in form.cleaned_data() as required
            # ...
            # redirect to. a new url
            return HttpResponseRedirect('/thanks/')
    # if GET or any other method , we will create a new form
    form = NameForm()
    
    return render(request, 'name.html', {form: 'form'})
```

If the form is submitted using a **POST** request, the view will once again create a form instance and populate it with data from the request: **form = NameForm(request.POST)** This is called **“binding data to the form”** (it is now a bound form).

We call the form’s **is_valid()** method; if it’s not True, we go back to the template with the form. This time the form is no longer empty (unbound) so the HTML form will be populated with the data previously submitted, where it can be edited and corrected as required.

**name.html**
``` html
<form action='/your_name/' method='POST'>
    {% csrf_token %}
    {{ form }}
    <input type='submit' name='Submit'>
</form>
```


        

## More about Django Form Class

Consider more useful example which can be used to implement contact me feature 

**forms.py**
``` Python
from django import forms

class ContactForm(forms.Form):
    subject = forms.CharField(max_length=100)
    message = forms.CharField(widget=Forms.Textarea)
    sender = forms.EmailField()
    cc_myself = forms.BooleanField(required=False)
    
```

Whatever the data submitted with a form, once it has been successfully validated by calling is_valid() (and is_valid() has returned True), the validated form data will be in the form.cleaned_data dictionary. This data will have been nicely converted into Python types for you.

**views.py**
``` python
from django.core.mail import send_mail

if form.is_valid():
    subject = form.cleaned_data['subject']
    message = form.cleaned_data['message']
    sender = form.cleaned_data['sender']
    cc_myself = form.cleaned_data['cc_myself']

    recipients = ['info@example.com']
    if cc_myself:
        recipients.append(sender)

    send_mail(subject, message, sender, recipients)
    return HttpResponseRedirect('/thanks/')
```

> **Tip**  
> For more on sending mail from django, see [sending mail](https://docs.djangoproject.com/en/2.1/topics/email/) 


## Working with form templates

### Form rendering option

There are other output options though for the `<label`>/`<input`> pairs:

* {{ form.as_table }} will render them as table cells wrapped in `<tr`> tags
* {{ form.as_p }} will render them wrapped in `<p`> tags
* {{ form.as_ul }} will render them wrapped in `<li`> tags  

Note that you’ll have to provide the surrounding `<table`> or `<ul`> elements yourself.

See [Outputting forms as html](https://docs.djangoproject.com/en/2.1/ref/forms/api/#ref-forms-api-outputting-html)for more eexample

### Rendering Fields Manually:
We don’t have to let Django unpack the form’s fields; we can do it manually if we like (allowing us to reorder the fields, for example). Each field is available as an attribute of the form using {{ form.name_of_field }}, and in a Django template, will be rendered appropriately. For example:

``` html
{{ form.non_field_errors }}
<div class="fieldWrapper">
    {{ form.subject.errors }}
    <label for="{{ form.subject.id_for_label }}">Email subject:</label>
    {{ form.subject }}
</div>
<div class="fieldWrapper">
    {{ form.message.errors }}
    <label for="{{ form.message.id_for_label }}">Your message:</label>
    {{ form.message }}
</div>
<div class="fieldWrapper">
    {{ form.sender.errors }}
    <label for="{{ form.sender.id_for_label }}">Your email address:</label>
    {{ form.sender }}
</div>
<div class="fieldWrapper">
    {{ form.cc_myself.errors }}
    <label for="{{ form.cc_myself.id_for_label }}">CC yourself?</label>
    {{ form.cc_myself }}
</div>
```

### Rendering form's error message
Using {{ form.name_of_field.errors }} displays a list of form errors, rendered as an unordered list. This might look like:

```html
<ul class="errorlist">
    <li>Sender is required.</li>
</ul>
```
The list has a CSS class of errorlist to allow you to style its appearance. If you wish to further customize the display of errors you can do so by looping over them:
```html
{% if form.subject.errors %}
    <ol>
    {% for error in form.subject.errors %}
        <li><strong>{{ error|escape }}</strong></li>
    {% endfor %}
    </ol>
{% endif %}
```
Non-field errors (and/or hidden field errors that are rendered at the top of the form when using helpers like form.as_p()) will be rendered with an additional class of nonfield to help distinguish them from field-specific errors. For example, {{ form.non_field_errors }} would look like:
```html
<ul class="errorlist nonfield">
    <li>Generic validation error</li>
</ul>
```
See [The Forms API](https://docs.djangoproject.com/en/2.1/ref/forms/api/) for more on errors, styling, and working with form attributes in templates.



### Looping over form's fields

If you’re using the same HTML for each of your form fields, you can reduce duplicate code by looping through each field in turn using a {% for %} loop:

``` html
{% for field in form %}
    <div class="fieldWrapper">
        {{ field.errors }}
        {{ field.label_tag }} {{ field }}
        {% if field.help_text %}
        <p class="help">{{ field.help_text|safe }}</p>
        {% endif %}
    </div>
{% endfor %}
```



## Core Field Arguments

Some important Fied arguments:

### required
By default, each Field class assumes the value is required, so if you pass an empty value – either None or the empty string ("") – then clean() will raise a ValidationError exception:

To specify that a field is not required, pass required=False to the Field constructor:
``` Python
> f = forms.CharField(required=False)
```

### label
The label argument lets you specify the “human-friendly” label for this field. This is used when the Field is displayed in a Form.

``` python
>> from django import forms
>> class CommentForm(forms.Form):
    name = forms.CharField(label='Your Name')
    url = forms.URLField(label='Your Website', required=False)
    comment = forms.CharField()
>> f = CommentForm(auto_id=False)
>> print(f)
<tr><th>Your name:</th><td><input type="text" name="name" required></td></tr>
<tr><th>Your website:</th><td><input type="url" name="url"></td></tr>
<tr><th>Comment:</th><td><input type="text" name="comment" required></td></tr>
 
```

### initial
The use-case for this is when you want to display an “empty” form in which a field is initialized to a particular value. For example:
``` python
>> from django import forms
>> class CommentForm(form.Form):
    name = forms.CharField(initial='Your Name')
    url = forms.UrlField(initial='http://')
    comment = forms.CharField()

>> f = CommentForm(auto_id='id_for_%s')
>> print(f)
<tr><th>Name:</th><td><input type="text" for='id_for_name' name="name" value="Your name" required></td></tr>
<tr><th>Url:</th><td><input type="url" for='id_for_url' name="url" value="http://" required></td></tr>
<tr><th>Comment:</th><td><input type="text" for='id_for_comment' name="comment" required></td></tr>
```

You may be thinking, why not just pass a dictionary of the initial values as data when displaying the form? Well, if you do that, you’ll trigger validation, and the HTML output will include any validation errors:
see [here](https://docs.djangoproject.com/en/2.1/ref/forms/fields/#django.forms.Field.initial)

Also note that initial values are not used as “fallback” data in validation if a particular field’s value is not given. initial values are only intended for initial form display.

### help_text
The help_text argument lets you specify descriptive text for this Field. If you provide help_text, it will be displayed next to the Field when the Field is rendered by one of the convenience Form methods (e.g., as_ul()).

### error_messages
The error_messages argument lets you override the default messages that the field will raise. Pass in a dictionary with keys matching the error messages you want to override. For example, here is the default error message:

``` python
>>> name = forms.CharField(error_messages={'required': 'Please enter your name'})

```

### validators
The validators argument lets you provide a list of validation functions for this field.
For more infor go [here](https://docs.djangoproject.com/en/2.1/ref/validators/)

### localize
The localize argument enables the localization of form data input, as well as the rendered output.
See the [format localization](https://docs.djangoproject.com/en/2.1/topics/i18n/formatting/) for more info.

### disabled
The disabled boolean argument, when set to True, disables a form field using the disabled HTML attribute so that it won’t be editable by users. Even if a user tampers with the field’s value submitted to the server, it will be ignored in favor of the value from the form’s initial data.




## Widgets
[see](https://docs.djangoproject.com/en/2.1/ref/forms/widgets/#built-in-widgets)

A widget is Django’s representation of an HTML input element. The widget handles the rendering of the HTML, and the extraction of data from a GET/POST dictionary that corresponds to the widget.

Example
```python
from django import forms

BIRTH_YEAR_CHOICES = ('1980', '1981', '1982')
FAVORITE_COLORS_CHOICES = (
    ('blue', 'Blue'),
    ('green', 'Green'),
    ('black', 'Black'),
)

class SimpleForm(forms.Form):
    birth_year = forms.DateField(widget=forms.SelectDateWidget(years=BIRTH_YEAR_CHOICES))
    favorite_colors = forms.MultipleChoiceField(
        required=False,
        widget=forms.CheckboxSelectMultiple,
        choices=FAVORITE_COLORS_CHOICES,
    )
```
See the [Built-in widgets](https://docs.djangoproject.com/en/2.1/ref/forms/widgets/#built-in-widgets) for more information about which widgets are available and which arguments they accept.

### Widgets inheriting from the Select widget

Widgets inheriting from the Select widget deal with choices. They present the user with a list of options to choose from. The different widgets present this choice differently; the Select widget itself uses a `<select`> HTML list representation, while RadioSelect uses radio buttons.

Select widgets are used by default on ChoiceField fields. The choices displayed on the widget are inherited from the ChoiceField and changing ChoiceField.choices will update Select.choices. For example:
```python
>>> from django import forms
>>> CHOICES = (('1', 'First',), ('2', 'Second',))
>>> choice_field = forms.ChoiceField(widget=forms.RadioSelect, choices=CHOICES)
>>> choice_field.choices
[('1', 'First'), ('2', 'Second')]
>>> choice_field.widget.choices
[('1', 'First'), ('2', 'Second')]
>>> choice_field.widget.choices = ()
>>> choice_field.choices = (('1', 'First and only',),)
>>> choice_field.widget.choices
[('1', 'First and only')]
```

### Styling widget instances

use the Widget.attrs argument when creating the widget to add some special css properties

```python
class CommentForm(forms.Form):
    name = forms.CharField(widget=forms.TextInput(attrs={'class': 'special'}))
    url = forms.URLField()
    comment = forms.CharField(widget=forms.TextInput(attrs={'size': '40'}))
```

You can also modify a widget in the form definition:

```python
class CommentForm(forms.Form):
    name = forms.CharField()
    url = forms.URLField()
    comment = forms.CharField()

    name.widget.attrs.update({'class': 'special'})
    comment.widget.attrs.update(size='40')
```

Or if the field isn’t declared directly on the form (such as model form fields), you can use the Form.fields attribute:

```python
class CommentForm(forms.ModelForm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fields['name'].widget.attrs.update({'class': 'special'})
        self.fields['comment'].widget.attrs.update(size='40')
 ```
More one styling widgets:
1. [Define a media inner class](https://docs.djangoproject.com/en/2.1/topics/forms/media/#assets-as-a-static-definition)
2. [Create a media property](https://docs.djangoproject.com/en/2.1/topics/forms/media/#dynamic-property)
3. [Form Assets](https://docs.djangoproject.com/en/2.1/topics/forms/media/)

### Multi Widget
[see](https://docs.djangoproject.com/en/2.1/ref/forms/widgets/#multiwidget)
Here’s an example widget which subclasses MultiWidget to display a date with the day, month, and year in different select boxes. This widget is intended to be used with a DateField rather than a MultiValueField, thus we have implemented value_from_datadict():

```python
from datetime import date
from django.forms import widgets

class DateSelectorWidget(widgets.MultiWidget):
    def __init__(self, attrs=None):
        # create choices for days, months, years
        # example below, the rest snipped for brevity.
        years = [(year, year) for year in (2011, 2012, 2013)]
        _widgets = (
            widgets.Select(attrs=attrs, choices=days),
            widgets.Select(attrs=attrs, choices=months),
            widgets.Select(attrs=attrs, choices=years),
        )
        super().__init__(_widgets, attrs)

    def decompress(self, value):
        if value:
            return [value.day, value.month, value.year]
        return [None, None, None]

    def value_from_datadict(self, data, files, name):
        datelist = [
            widget.value_from_datadict(data, files, name + '_%s' % i)
            for i, widget in enumerate(self.widgets)]
        try:
            D = date(
                day=int(datelist[0]),
                month=int(datelist[1]),
                year=int(datelist[2]),
            )
        except ValueError:
            return ''
        else:
            return str(D)
```


## Creating Forms from Model
[see](https://docs.djangoproject.com/en/2.1/topics/forms/modelforms/)

### A full example:
Consider following set of models

``` python

from django.db import models
from django.forms import ModelForm

TITLE_CHOICES = (
    ('MR', 'Mr.'),
    ('MRS', 'Mrs.'),
    ('MS', 'Ms.'),
)

class Author(models.Model):
    name = models.CharField(max_length=100)
    title = models.CharField(max_length=3, choices=TITLE_CHOICES)
    birth_date = models.DateField(blank=True, null=True)

    def __str__(self):
        return self.name

class Book(models.Model):
    name = models.CharField(max_length=100)
    authors = models.ManyToManyField(Author)

class AuthorForm(ModelForm):
    class Meta:
        model = Author
        fields = ['name', 'title', 'birth_date']

class BookForm(ModelForm):
    class Meta:
        model = Book
        fields = ['name', 'authors']
```

With these models, the ModelForm subclasses above would be roughly equivalent to this (the only difference being the save() method, which we’ll discuss in a moment.):

``` python
from django import forms

class AuthorForm(forms.Form):
    name = forms.CharField(max_length=100)
    title = forms.CharField(
        max_length=3,
        widget=forms.Select(choices=TITLE_CHOICES),
    )
    birth_date = forms.DateField(required=False)

class BookForm(forms.Form):
    name = forms.CharField(max_length=100)
    authors = forms.ModelMultipleChoiceField(queryset=Author.objects.all())
```


### The save() method

This method creates and saves a database object from the data bound to the form. A subclass of ModelForm can accept an existing model instance as the keyword argument instance; if this is supplied, save() will update that instance. If it’s not supplied, save() will create a new instance of the specified model:

```python
>> from myapp.models import Article
>> from myapp.forms import ArticleForm

# create a form instance from form post data
>> f = ArticleForm(request.POST)

# save a new_article form from form data
>> new_article = f.save()

# crrate a form to edit an existing article, but use
# POST data to populate form
>> a = Article.objects.get(pk=1)
>> f = ArticleForm(request.POST, instance=a)
>> f.save()
```

This save() method accepts an optional commit keyword argument, which accepts either True or False. If you call save() with commit=False, then it will return an object that hasn’t yet been saved to the database. In this case, it’s up to you to call save() on the resulting model instance. This is useful if you want to do custom processing on the object before saving it, or if you want to use one of the specialized model saving options. commit is True by default.

Another side effect of using commit=False is seen when your model has a many-to-many relation with another model. If your model has a many-to-many relation and you specify commit=False when you save a form, Django cannot immediately save the form data for the many-to-many relation. This is because it isn’t possible to save many-to-many data for an instance until the instance exists in the database.

To work around this problem, every time you save a form using commit=False, Django adds a save_m2m() method to your ModelForm subclass. After you’ve manually saved the instance produced by the form, you can invoke save_m2m() to save the many-to-many form data. For example:

```python
# Create a form instance with POST data.
>>> f = AuthorForm(request.POST)

# Create, but don't save the new author instance.
>>> new_author = f.save(commit=False)

# Modify the author in some way.
>>> new_author.some_field = 'some_value'

# Save the new instance.
>>> new_author.save()

# Now, save the many-to-many data for the form.
>>> f.save_m2m()
```

> **Note**   
> Calling save_m2m() is only required if you use save(commit=False). When you use a simple save() on a form, all data – including many-to-many data – is saved without the need for any additional method calls.

### Selecting Fields to Use

1. Set the fields attribute to the special value '__all__' to indicate that all fields in the model should be used.   
    For example:

```python
from django.forms import ModelForm

class AuthorForm(ModelForm):
    class Meta:
        model = Author
        fields = '__all__'
```

2. Set the exclude attribute of the ModelForm’s inner Meta class to a list of fields to be excluded from the form.  
    For example:
```python
class PartialAuthorForm(ModelForm):
    class Meta:
        model = Author
        exclude = ['title']
```

> **Note**  
> Any fields not included in a form by the above logic will not be set by the form’s save() method. Also, if you manually add the excluded fields back to the form, they will not be initialized from the model instance.
>
> Django will prevent any attempt to save an incomplete model, so if the model does not allow the missing fields to be empty, and does not provide a default value for the missing fields, any attempt to save() a ModelForm with missing fields will fail. To avoid this failure, you must instantiate your model with initial values for the missing, but required fields:
> 
> ``` python
    author = Author(title='Mr')
    form = PartialAuthorForm(request.POST, instance=author)
    form.save()
> ```
> Alternatively, you can use save(commit=False) and manually set any extra required fields:
> ```python
    form = PartialAuthorForm(request.POST)
    author = form.save(commit=False)
    author.title = 'Mr'
    author.save()
> ```



### Overriding the default fields

ModelForm gives you the flexibility of changing the form field for a given model.

For example, if you want the CharField for the name attribute of Author to be represented by a `<textarea`> instead of its default `<input type="text"`>, you can override the field’s widget:

``` python
from django.forms import ModelForm, Textarea
from myapp.models import Author

class AuthorForm(ModelForm):
    class Meta:
        model = Author
        fields = ('name', 'title', 'birth_date')
        widgets = {
            'name': Textarea(attrs={'cols': 80, 'rows': 20}),
        }
```

Similarly, you can specify the labels, help_texts and error_messages attributes of the inner Meta class if you want to further customize a field.

For example if you wanted to customize the wording of all user facing strings for the name field:

```python
from django.utils.translation import gettext_lazy as _

class AuthorForm(ModelForm):
    class Meta:
        model = Author
        fields = ('name', 'title', 'birth_date')
        labels = {
            'name': _('Writer'),
        }
        help_texts = {
            'name': _('Some useful help text.'),
        }
        error_messages = {
            'name': {
                'max_length': _("This writer's name is too long."),
            },
        }
```

### Providing initial values

As with regular forms, it’s possible to specify initial data for forms by specifying an initial parameter when instantiating the form. Initial values provided this way will override both initial values from the form field and values from an attached model instance. For example:

```python
>>> article = Article.objects.get(pk=1)
>>> article.headline
'My headline'
>>> form = ArticleForm(initial={'headline': 'Initial headline'}, instance=article)
>>> form['headline'].value()
'Initial headline'
```

### ModelForm factory function

You can create forms from a given model using the standalone function modelform_factory(), instead of using a class definition. This may be more convenient if you do not have many customizations to make:

``` python
>>> from django.forms import modelform_factory
>>> from myapp.models import Book
>>> BookForm = modelform_factory(Book, fields=("author", "title"))
```

This can also be used to make simple modifications to existing forms, for example by specifying the widgets to be used for a given field:

```python
>>> from django.forms import Textarea
>>> Form = modelform_factory(Book, form=BookForm,
...                          widgets={"title": Textarea()})
```
The fields to include can be specified using the fields and exclude keyword arguments, or the corresponding attributes on the ModelForm inner Meta class. Please see the ModelForm [Selecting the fields to use](https://docs.djangoproject.com/en/2.1/topics/forms/modelforms/#modelforms-selecting-fields) documentation.

… or enable localization for specific fields:

```python
>>> Form = modelform_factory(Author, form=AuthorForm, localized_fields=("birth_date",))
```



## Formsets
[see](https://docs.djangoproject.com/en/2.1/topics/forms/formsets/)

Let’s say you have the following form:

```python
>>> from django import forms
>>> class ArticleForm(forms.Form):
...     title = forms.CharField()
...     pub_date = forms.DateField()
``` 

You might want to allow the user to create several articles at once. To create a formset out of an ArticleForm you would do:

```python
>>> from django.forms import formset_factory
>>> ArticleFormSet = formset_factory(ArticleForm, extra=2)
```

### Using initial data with formsets

```python
>>> ArticleFormSet = formset_factory(ArticleForm, extra=2)
>>> formset = ArticleFormSet(initial=[
...     {'title': 'Django is now open source',
...      'pub_date': datetime.date.today(),}
... ])
```

### Custom Formset Validation
A formset has a clean method similar to the one on a Form class. This is where you define your own validation that works at the formset level:

```python
>>> from django.forms import BaseFormSet
>>> from django.forms import formset_factory
>>> from myapp.forms import ArticleForm

>>> class BaseArticleFormSet(BaseFormSet):
...     def clean(self):
...         """Checks that no two articles have the same title."""
...         if any(self.errors):
...             # Don't bother validating the formset unless each form is valid on its own
...             return
...         titles = []
...         for form in self.forms:
...             title = form.cleaned_data['title']
...             if title in titles:
...                 raise forms.ValidationError("Articles in a set must have distinct titles.")
...             titles.append(title)

>>> ArticleFormSet = formset_factory(ArticleForm, formset=BaseArticleFormSet)
>>> data = {
...     'form-TOTAL_FORMS': '2',
...     'form-INITIAL_FORMS': '0',
...     'form-MAX_NUM_FORMS': '',
...     'form-0-title': 'Test',
...     'form-0-pub_date': '1904-06-16',
...     'form-1-title': 'Test',
...     'form-1-pub_date': '1912-06-23',
... }
>>> formset = ArticleFormSet(data)
>>> formset.is_valid()
False
>>> formset.errors
[{}, {}]
>>> formset.non_form_errors()
['Articles in a set must have distinct titles.']
```
The formset clean method is called after all the Form.clean methods have been called. The errors will be found using the non_form_errors() method on the formset.

### can_delete

BaseFormSet.can_delete
Default: False

Lets you create a formset with the ability to select forms for deletion:
```python
>>> from django.forms import formset_factory
>>> from myapp.forms import ArticleForm
>>> ArticleFormSet = formset_factory(ArticleForm, can_delete=True)
```

### Adding additional fields to a formset

If you need to add additional fields to the formset this can be easily accomplished. The formset base class provides an add_fields method. You can simply override this method to add your own fields or even redefine the default fields/attributes of the order and deletion fields:

```python
>>> from django.forms import BaseFormSet
>>> from django.forms import formset_factory
>>> from myapp.forms import ArticleForm
>>> class BaseArticleFormSet(BaseFormSet):
...     def add_fields(self, form, index):
...         super().add_fields(form, index)
...         form.fields["my_field"] = forms.CharField()

>>> ArticleFormSet = formset_factory(ArticleForm, formset=BaseArticleFormSet)
```

### Using a formset in views and templates

Using a formset inside a view is as easy as using a regular Form class. The only thing you will want to be aware of is making sure to use the management form inside the template. Let’s look at a sample view:

```python
from django.forms import formset_factory
from django.shortcuts import render
from myapp.forms import ArticleForm

def manage_articles(request):
    ArticleFormSet = formset_factory(ArticleForm)
    if request.method == 'POST':
        formset = ArticleFormSet(request.POST, request.FILES)
        if formset.is_valid():
            # do something with the formset.cleaned_data
            pass
    else:
        formset = ArticleFormSet()
    return render(request, 'manage_articles.html', {'formset': formset})
```

The manage_articles.html template might look like this:

```html
<form method="post">
    {{ formset.management_form }}
    <table>
        {% for form in formset %}
        {{ form }}
        {% endfor %}
    </table>
</form>
```
However there’s a slight shortcut for the above by letting the formset itself deal with the management form:

```html
<form method="post">
    <table>
        {{ formset }}
    </table>
</form>
```

The above ends up calling the as_table method on the formset class.

### Manually rendered can_delete and can_order

If you manually render fields in the template, you can render can_delete parameter with {{ form.DELETE }}:

```html
<form method="post">
    {{ formset.management_form }}
    {% for form in formset %}
        <ul>
            <li>{{ form.title }}</li>
            <li>{{ form.pub_date }}</li>
            {% if formset.can_delete %}
                <li>{{ form.DELETE }}</li>
            {% endif %}
        </ul>
    {% endfor %}
</form>
```
Similarly, if the formset has the ability to order (can_order=True), it is possible to render it with {{ form.ORDER }}.

### Using more than one formset in a view

You are able to use more than one formset in a view if you like. Formsets borrow much of its behavior from forms. With that said you are able to use prefix to prefix formset form field names with a given value to allow more than one formset to be sent to a view without name clashing. Lets take a look at how this might be accomplished:

```python
from django.forms import formset_factory
from django.shortcuts import render
from myapp.forms import ArticleForm, BookForm

def manage_articles(request):
    ArticleFormSet = formset_factory(ArticleForm)
    BookFormSet = formset_factory(BookForm)
    if request.method == 'POST':
        article_formset = ArticleFormSet(request.POST, request.FILES, prefix='articles')
        book_formset = BookFormSet(request.POST, request.FILES, prefix='books')
        if article_formset.is_valid() and book_formset.is_valid():
            # do something with the cleaned_data on the formsets.
            pass
    else:
        article_formset = ArticleFormSet(prefix='articles')
        book_formset = BookFormSet(prefix='books')
    return render(request, 'manage_articles.html', {
        'article_formset': article_formset,
        'book_formset': book_formset,
    })
```

You would then render the formsets as normal. It is important to point out that you need to pass prefix on both the POST and non-POST cases so that it is rendered and processed correctly.



## Model Formsets
[see](https://docs.djangoproject.com/en/2.1/topics/forms/modelforms/#model-formsets)

```python
>>> from django.forms import modelformset_factory
>>> from myapp.models import Author
>>> AuthorFormSet = modelformset_factory(Author, fields=('name', 'title'))
```

Using fields restricts the formset to use only the given fields. Alternatively, you can take an “opt-out” approach, specifying which fields to exclude:

```python
>>> AuthorFormSet = modelformset_factory(Author, exclude=('birth_date',))
```

### Changing the queryset

By default, when you create a formset from a model, the formset will use a queryset that includes all objects in the model (e.g., Author.objects.all()). You can override this behavior by using the queryset argument:

```python
>>> formset = AuthorFormSet(queryset=Author.objects.filter(name__startswith='O'))
```

Alternatively, you can create a subclass that sets self.queryset in __init__:

```python
from django.forms import BaseModelFormSet
from myapp.models import Author

class BaseAuthorFormSet(BaseModelFormSet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.queryset = Author.objects.filter(name__startswith='O')
```

Then, pass your BaseAuthorFormSet class to the factory function:
```python
>>> AuthorFormSet = modelformset_factory(
...     Author, fields=('name', 'title'), formset=BaseAuthorFormSet)
```

If you want to return a formset that doesn’t include any pre-existing instances of the model, you can specify an empty QuerySet:
```python
>>> AuthorFormSet(queryset=Author.objects.none())
```

### Changing the form

By default, when you use modelformset_factory, a model form will be created using modelform_factory(). Often, it can be useful to specify a custom model form. For example, you can create a custom model form that has custom validation:

```python
class AuthorForm(forms.ModelForm):
    class Meta:
        model = Author
        fields = ('name', 'title')

    def clean_name(self):
        # custom validation for the name field
        ...
 ```
 
Then, pass your model form to the factory function:

```python
AuthorFormSet = modelformset_factory(Author, form=AuthorForm)
```
It is not always necessary to define a custom model form. The modelformset_factory function has several arguments which are passed through to modelform_factory, which are described below.

### Specifying widgets to use in the form with widgets

Using the widgets parameter, you can specify a dictionary of values to customize the ModelForm’s widget class for a particular field. This works the same way as the widgets dictionary on the inner Meta class of a ModelForm works:
```python
>>> AuthorFormSet = modelformset_factory(
...     Author, fields=('name', 'title'),
...     widgets={'name': Textarea(attrs={'cols': 80, 'rows': 20})})
```

### Saving objects in the formset

As with a ModelForm, you can save the data as a model object. This is done with the formset’s save() method:

```python

# Create a formset instance with POST data.
>>> formset = AuthorFormSet(request.POST)

# Assuming all is valid, save the data.
>>> instances = formset.save()
```
The save() method returns the instances that have been saved to the database. If a given instance’s data didn’t change in the bound data, the instance won’t be saved to the database and won’t be included in the return value (instances, in the above example).

When fields are missing from the form (for example because they have been excluded), these fields will not be set by the save() method. You can find more information about this restriction, which also holds for regular ModelForms, in Selecting the fields to use.

Pass commit=False to return the unsaved model instances:

```python
# don't save to the database
>>> instances = formset.save(commit=False)
>>> for instance in instances:
...     # do something with instance
...     instance.save()
```
This gives you the ability to attach data to the instances before saving them to the database. If your formset contains a ManyToManyField, you’ll also need to call formset.save_m2m() to ensure the many-to-many relationships are saved properly.

### Using a model formset in a view

Model formsets are very similar to formsets. Let’s say we want to present a formset to edit Author model instances:
```python
from django.forms import modelformset_factory
from django.shortcuts import render
from myapp.models import Author

def manage_authors(request):
    AuthorFormSet = modelformset_factory(Author, fields=('name', 'title'))
    if request.method == 'POST':
        formset = AuthorFormSet(request.POST, request.FILES)
        if formset.is_valid():
            formset.save()
            # do something.
    else:
        formset = AuthorFormSet()
    return render(request, 'manage_authors.html', {'formset': formset})
```
As you can see, the view logic of a model formset isn’t drastically different than that of a “normal” formset. The only difference is that we call formset.save() to save the data into the database.